In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from PIL import Image
from joblib import dump, load

%matplotlib inline

In [2]:
# Importing training data.
data_train = pd.read_csv('mnist_train.csv')
len(data_train)

60000

In [3]:
# Splitting into features and labels.
x_train = data_train.drop('label', axis=1).astype('float32')/255
y_train = data_train['label']
len(x_train), len(y_train)

(60000, 60000)

In [4]:
# Finding out the unique labels.
unique_labels = np.unique(y_train)
unique_labels

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)

In [5]:
def encode_labels(y):
    """
    This function converts the labels into their
    one-hot encoded equivalent.
    """
    y_enc = []
    for i in range(len(y)):
        y_enc.append((y[i] == unique_labels).astype(int))
        
    return np.array(y_enc)

In [6]:
# Encoding the training labels.
y_train_encoded = encode_labels(y_train)
y_train_encoded

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [267]:
# Instantiate model.
model = MLPClassifier(verbose=True,
                      random_state=10,
                      shuffle=True,
                      early_stopping=True,
                      max_iter=500)

# Fit data to the model.
model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.34580704
Validation score: 0.805833
Iteration 2, loss = 0.62875170
Validation score: 0.857667
Iteration 3, loss = 0.48572432
Validation score: 0.885833
Iteration 4, loss = 0.39265251
Validation score: 0.903833
Iteration 5, loss = 0.32961198
Validation score: 0.912167
Iteration 6, loss = 0.28377151
Validation score: 0.924000
Iteration 7, loss = 0.25076370
Validation score: 0.926500
Iteration 8, loss = 0.22515969
Validation score: 0.933167
Iteration 9, loss = 0.20533962
Validation score: 0.932167
Iteration 10, loss = 0.18786829
Validation score: 0.938333
Iteration 11, loss = 0.17304292
Validation score: 0.936333
Iteration 12, loss = 0.15933087
Validation score: 0.939667
Iteration 13, loss = 0.14766584
Validation score: 0.940167
Iteration 14, loss = 0.13606382
Validation score: 0.939000
Iteration 15, loss = 0.12645194
Validation score: 0.943000
Iteration 16, loss = 0.11772558
Validation score: 0.944500
Iteration 17, loss = 0.10935074
Validation score: 0.942500
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [268]:
# Saving trained model.
dump(model, 'digit_classifier_t1.joblib')

['digit_classifier_t1.joblib']

In [18]:
# Loading saved model.
model = load('digit_classifier_t1.joblib')

In [7]:
# Importing test data.
data_test = pd.read_csv('mnist_test.csv')
len(data_test)

10000

In [8]:
# Splitting into features and labels.
x_test = data_test.drop('label', axis=1).astype('float32')/255
y_test = data_test['label']
len(x_test), len(y_test)

(10000, 10000)

In [9]:
# Encoding the test labels.
y_test_encoded = encode_labels(y_test)
y_test_encoded

array([[0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [11]:
# Scoring the model.
model.score(x_test, y_test_encoded)

0.9584

In [10]:
def process_pred(pred):
    """
    This function returnns the digit predicted
    by the model.
    """
    return np.argmax(pred)

In [11]:
def resize_center_image(old_im):
    old_im = old_im.resize((20,20))
    old_size = old_im.size

    new_size = (28, 28)
    new_im = Image.new("L", new_size) 
    new_im.paste(old_im, (4,4))

    return new_im

In [12]:
def process_image(image_path):
    """
    Processes the given image.
    """
    im = Image.open(image_path)
    im = resize_center_image(im)
    return pd.DataFrame(np.asarray(im).flatten().astype(float)/255).T

In [13]:
def show_image(image_path):
    """
    Displays processed image.
    """
    im = Image.open(image_path)
    im = resize_center_image(im)
    return im

In [46]:
IMG_PATH = 'test_img/test_3.png'

In [47]:
show_image(IMG_PATH)

In [48]:
process_image(IMG_PATH)

,0,1,2,3,4,5,6,7,8,9,...,774,775,776,777,778,779,780,781,782,783
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
pred = model.predict(process_image(IMG_PATH))

In [50]:
process_pred(pred)

0

In [23]:
model.n_layers_

3

In [21]:
# Trying with 2 hidden layers with 64 neurons each.
rs_model = MLPClassifier(verbose=True,
                         random_state=10,
                         shuffle=True,
                         early_stopping=True,
                         max_iter=500,
                         hidden_layer_sizes=(64,64))

rs_model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.31353039
Validation score: 0.867000
Iteration 2, loss = 0.45960701
Validation score: 0.901833
Iteration 3, loss = 0.35331626
Validation score: 0.917167
Iteration 4, loss = 0.28907934
Validation score: 0.928500
Iteration 5, loss = 0.24287087
Validation score: 0.935333
Iteration 6, loss = 0.20555253
Validation score: 0.943667
Iteration 7, loss = 0.17571264
Validation score: 0.948333
Iteration 8, loss = 0.15413013
Validation score: 0.949500
Iteration 9, loss = 0.13789178
Validation score: 0.949167
Iteration 10, loss = 0.12147289
Validation score: 0.954000
Iteration 11, loss = 0.10922982
Validation score: 0.956667
Iteration 12, loss = 0.09765925
Validation score: 0.958167
Iteration 13, loss = 0.08774177
Validation score: 0.957500
Iteration 14, loss = 0.07996434
Validation score: 0.955000
Iteration 15, loss = 0.07241579
Validation score: 0.959667
Iteration 16, loss = 0.06492255
Validation score: 0.960500
Iteration 17, loss = 0.05838673
Validation score: 0.960500
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(64, 64), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [22]:
rs_model.score(x_test, y_test_encoded)

0.9685

In [23]:
# Trying with 3 hidden layers of 64 neurons each.
rs2_model = MLPClassifier(verbose=True,
                         random_state=10,
                         shuffle=True,
                         early_stopping=True,
                         max_iter=500,
                         hidden_layer_sizes=(64,64,64))

rs2_model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.25031411
Validation score: 0.881167
Iteration 2, loss = 0.39774475
Validation score: 0.914500
Iteration 3, loss = 0.28750595
Validation score: 0.932667
Iteration 4, loss = 0.23771375
Validation score: 0.938667
Iteration 5, loss = 0.19866518
Validation score: 0.945333
Iteration 6, loss = 0.17428167
Validation score: 0.947667
Iteration 7, loss = 0.15193380
Validation score: 0.953500
Iteration 8, loss = 0.13335148
Validation score: 0.955333
Iteration 9, loss = 0.11977912
Validation score: 0.953833
Iteration 10, loss = 0.10611787
Validation score: 0.957667
Iteration 11, loss = 0.09743947
Validation score: 0.956833
Iteration 12, loss = 0.08625189
Validation score: 0.958833
Iteration 13, loss = 0.07766887
Validation score: 0.960167
Iteration 14, loss = 0.07090006
Validation score: 0.960500
Iteration 15, loss = 0.06351310
Validation score: 0.958500
Iteration 16, loss = 0.05460646
Validation score: 0.959000
Iteration 17, loss = 0.05223510
Validation score: 0.960500
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(64, 64, 64), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [24]:
rs2_model.score(x_test, y_test_encoded)

0.9662

In [32]:
# Trying with 2 hidden layers of 128 neurons each.
rs3_model = MLPClassifier(verbose=True,
                         random_state=10,
                         shuffle=True,
                         early_stopping=True,
                         max_iter=500,
                         hidden_layer_sizes=(128,128))

rs3_model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.15164973
Validation score: 0.879333
Iteration 2, loss = 0.38877370
Validation score: 0.919500
Iteration 3, loss = 0.27602625
Validation score: 0.937000
Iteration 4, loss = 0.21478609
Validation score: 0.946333
Iteration 5, loss = 0.17170477
Validation score: 0.954667
Iteration 6, loss = 0.14275587
Validation score: 0.957667
Iteration 7, loss = 0.12000565
Validation score: 0.958333
Iteration 8, loss = 0.10137834
Validation score: 0.959000
Iteration 9, loss = 0.08649727
Validation score: 0.963667
Iteration 10, loss = 0.07277177
Validation score: 0.964500
Iteration 11, loss = 0.06113390
Validation score: 0.963167
Iteration 12, loss = 0.05224151
Validation score: 0.963833
Iteration 13, loss = 0.04461838
Validation score: 0.966333
Iteration 14, loss = 0.03655666
Validation score: 0.966833
Iteration 15, loss = 0.03066144
Validation score: 0.966833
Iteration 16, loss = 0.02566038
Validation score: 0.965500
Iteration 17, loss = 0.02210491
Validation score: 0.969667
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(128, 128), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [33]:
rs3_model.score(x_test, y_test_encoded)

0.9715

In [34]:
# Trying with 1 hidden layer with 256 neurons.
rs4_model = MLPClassifier(verbose=True,
                         random_state=10,
                         shuffle=True,
                         early_stopping=True,
                         max_iter=500,
                         hidden_layer_sizes=(256))

rs4_model.fit(x_train, y_train_encoded)

Iteration 1, loss = 1.00483010
Validation score: 0.863500
Iteration 2, loss = 0.42865584
Validation score: 0.911500
Iteration 3, loss = 0.31571957
Validation score: 0.925833
Iteration 4, loss = 0.25139626
Validation score: 0.941167
Iteration 5, loss = 0.20938517
Validation score: 0.947000
Iteration 6, loss = 0.17679760
Validation score: 0.952667
Iteration 7, loss = 0.15200095
Validation score: 0.955000
Iteration 8, loss = 0.13335590
Validation score: 0.959833
Iteration 9, loss = 0.11649757
Validation score: 0.959333
Iteration 10, loss = 0.10237136
Validation score: 0.960667
Iteration 11, loss = 0.09012821
Validation score: 0.961500
Iteration 12, loss = 0.07989210
Validation score: 0.962833
Iteration 13, loss = 0.06972146
Validation score: 0.963667
Iteration 14, loss = 0.06062473
Validation score: 0.964833
Iteration 15, loss = 0.05434749
Validation score: 0.961667
Iteration 16, loss = 0.04797836
Validation score: 0.965000
Iteration 17, loss = 0.04173196
Validation score: 0.965167
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=256, learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [35]:
rs4_model.score(x_test, y_test_encoded)

0.9653

In [36]:
# Trying with 2 hidden layers of 256 neurons each.
rs5_model = MLPClassifier(verbose=True,
                         random_state=10,
                         shuffle=True,
                         early_stopping=True,
                         max_iter=500,
                         hidden_layer_sizes=(256,256))

rs5_model.fit(x_train, y_train_encoded)

Iteration 1, loss = 0.87889413
Validation score: 0.915500
Iteration 2, loss = 0.28682241
Validation score: 0.944667
Iteration 3, loss = 0.19005111
Validation score: 0.956167
Iteration 4, loss = 0.13739519
Validation score: 0.960000
Iteration 5, loss = 0.10355133
Validation score: 0.962667
Iteration 6, loss = 0.07971110
Validation score: 0.964500
Iteration 7, loss = 0.06242294
Validation score: 0.969500
Iteration 8, loss = 0.04857640
Validation score: 0.968000
Iteration 9, loss = 0.03702180
Validation score: 0.967167
Iteration 10, loss = 0.02713969
Validation score: 0.968833
Iteration 11, loss = 0.02209002
Validation score: 0.966667
Iteration 12, loss = 0.01864678
Validation score: 0.968833
Iteration 13, loss = 0.01344290
Validation score: 0.970500
Iteration 14, loss = 0.01041717
Validation score: 0.968333
Iteration 15, loss = 0.00845461
Validation score: 0.970833
Iteration 16, loss = 0.00741134
Validation score: 0.971000
Iteration 17, loss = 0.01034129
Validation score: 0.969167
Iterat

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(256, 256), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=500,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=10, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [37]:
rs5_model.score(x_test, y_test_encoded)

0.9762

In [38]:
dump(rs_model, 'digit_classifier_t2_6464.joblib')

['digit_classifier_t2_6464.joblib']

In [39]:
dump(rs3_model, 'digit_classifier_t2_128128.joblib')

['digit_classifier_t2_128128.joblib']

In [40]:
dump(rs5_model, 'digit_classifier_t2_256256.joblib')

['digit_classifier_t2_256256.joblib']

In [90]:
IMG_PATH = 'test_img/test.png'

In [157]:
show_image(IMG_PATH)

In [158]:
process_pred(rs5_model.predict(process_image(IMG_PATH)))

0

In [159]:
process_pred(model.predict(process_image(IMG_PATH)))

9